In [1]:
import ccxt
import math
from datetime import datetime
import numpy as np
import re

In [2]:
def fetch_exchange(exch_name, exch):
    graph = {}
    # load markets
    market = exch.load_markets(True)

    if (exch.has['fetchTickers']):
        exch_tickers = exch.fetch_tickers()
        for symbol in exch_tickers.keys():
            try:
                node_to, node_from = symbol.split('/')
                try:
                    w_to = -math.log(1 / float(exch_tickers[symbol]['info']['askPrice']))
                    w_from = -math.log(float(exch_tickers[symbol]['info']['askPrice']))
                    if node_from not in graph:
                        graph[node_from] = {}
                    graph[node_from][node_to] = {'weight': w_to, 'd': 'direct'}
                    if node_to not in graph:
                        graph[node_to] = {}
                    graph[node_to][node_from] = {'weight': w_from, 'd': 'reverse'}
                except:
                    pass
            except:
                print('symbol error')
    return graph

In [3]:
def initialize(graph, source):
    d = {} # Stands for destination
    p = {} # Stands for predecessor
    for node in graph:
        d[node] = float('Inf') # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0 # For the source we know how to reach
    return d, p
 
def relax(node, neighbour, graph, d, p):
    fee = -math.log(1 - 0.001)
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]['weight'] + fee:
        # Record this lower distance
        d[neighbour]  = d[node] + graph[node][neighbour]['weight'] + fee
        p[neighbour] = node
        
def retrace_negative_loop(p, start):
    
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1): #Run this until is converges
        for u in graph:
            for v in graph[u]: #For each neighbour of u
                relax(u, v, graph, d, p) #Lets relax it


    # Step 3: check for negative-weight cycles
    fee = -math.log(1 - 0.001)
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]['weight'] + fee:
                return(retrace_negative_loop(p, source))
    return None


def collect_negative_cycle():
    binance = ccxt.binance({
    'apiKey': 'y',
    'secret': 'Y', })
    
    paths = []
    graph = fetch_exchange('binance', binance)
    print('graph collected')

    path = bellman_ford(graph, 'USDT')
    if path not in paths and not None:
        paths.append(path)

    for path in paths:
        if path == None:
            print("No opportunity here :(")
        else:
            print(path)
            graph_sum = 0
            for i in range(len(path) - 1):
                print(graph[path[i]][path[i + 1]]['weight'])
                print(graph[path[i + 1]][path[i]]['weight'])
                graph_sum += graph[path[i]][path[i + 1]]['weight']
                graph_sum += graph[path[i + 1]][path[i]]['weight']
            print('total sum:')
            print(graph_sum)

In [4]:
collect_negative_cycle()

graph collected
['USDT', 'NMR', 'BUSD', 'TRB', 'BTC', 'HOT', 'USDT']
3.5340368683082666
-3.5340368683082666
-3.531728444461926
3.531728444461926
3.5198097029957296
-3.5198097029957296
6.345286422064308
-6.345286422064308
-17.034386382832476
17.034386382832476
7.49186869851796
-7.49186869851796
total sum:
0.0
